In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
import matplotlib.pyplot as plt

In [66]:
data = 'CRSPDaily2015to20220331.csv'
df = pd.read_csv(
    data,
    dtype={'date': str},      # read date as literal text
    parse_dates=False         # don’t auto-parse dates
)

/var/folders/kp/mvqzzmf14fj_byq4km78jych0000gn/T/ipykernel_25527/1662202077.py:2: DtypeWarning: Columns (5,6,9,18,19,49,57) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [87]:
df.head()

,date,TICKER,PERMNO,PRC,log_return,MKT_RF,SMB,HML,RMW,CMA,RF,MOM
13962,2015-01-02,ORCL,10104,44.33,NaN,-0.0012,-0.0057,0.0008,-0.0025,0.0012,0.0,NaN
13963,2015-01-05,ORCL,10104,43.59,-0.016834,-0.0184,0.0025,-0.0068,0.0016,-0.0008,0.0,NaN
13964,2015-01-06,ORCL,10104,43.14,-0.010377,-0.0104,-0.0078,-0.0031,0.0053,0.0002,0.0,NaN
13965,2015-01-07,ORCL,10104,43.15,0.000232,0.0119,0.0017,-0.0066,0.0025,-0.0014,0.0,NaN
13966,2015-01-08,ORCL,10104,43.41,0.006007,0.0181,-0.0010,-0.0028,0.0011,-0.0016,0.0,NaN


In [88]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_table = pd.read_html(url)[0]
tickers = sp500_table["Symbol"].tolist()
tickers = [t.replace('.', '-') for t in tickers]

In [89]:
df_filtered = df[df['TICKER'].isin(tickers)]

In [90]:
df_filtered.head()

,date,TICKER,PERMNO,PRC,log_return,MKT_RF,SMB,HML,RMW,CMA,RF,MOM
13962,2015-01-02,ORCL,10104,44.33,NaN,-0.0012,-0.0057,0.0008,-0.0025,0.0012,0.0,NaN
13963,2015-01-05,ORCL,10104,43.59,-0.016834,-0.0184,0.0025,-0.0068,0.0016,-0.0008,0.0,NaN
13964,2015-01-06,ORCL,10104,43.14,-0.010377,-0.0104,-0.0078,-0.0031,0.0053,0.0002,0.0,NaN
13965,2015-01-07,ORCL,10104,43.15,0.000232,0.0119,0.0017,-0.0066,0.0025,-0.0014,0.0,NaN
13966,2015-01-08,ORCL,10104,43.41,0.006007,0.0181,-0.0010,-0.0028,0.0011,-0.0016,0.0,NaN


In [91]:
df.columns

Index(['date', 'TICKER', 'PERMNO', 'PRC', 'log_return', 'MKT_RF', 'SMB', 'HML',
       'RMW', 'CMA', 'RF', 'MOM'],
      dtype='object')

In [92]:
df_filtered['log_return'] = df_filtered.groupby('PERMNO')['PRC'].transform(lambda x: np.log(x / x.shift(1)))

/Users/pranshu/opt/anaconda3/envs/Formula/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [93]:
useful_cols = ['date', 'TICKER', 'PERMNO', 'PRC', 'log_return']
df_filtered = df_filtered[useful_cols]

In [94]:
df_filtered.head(20)

,date,TICKER,PERMNO,PRC,log_return
13962,2015-01-02,ORCL,10104,44.33,NaN
13963,2015-01-05,ORCL,10104,43.59,-0.016834
13964,2015-01-06,ORCL,10104,43.14,-0.010377
13965,2015-01-07,ORCL,10104,43.15,0.000232
13966,2015-01-08,ORCL,10104,43.41,0.006007
13967,2015-01-09,ORCL,10104,43.39,-0.000461
13968,2015-01-12,ORCL,10104,43.27,-0.002769
13969,2015-01-13,ORCL,10104,42.93,-0.007889
13970,2015-01-14,ORCL,10104,43.06,0.003024
13971,2015-01-15,ORCL,10104,42.63,-0.010036


In [95]:
mu = df_filtered['log_return'].mean()
sigma = df_filtered['log_return'].std()

lower_bound = mu - 2 * sigma
upper_bound = mu + 2 * sigma

df_filtered['log_return'] = df_filtered['log_return'].clip(lower_bound, upper_bound)


In [96]:
print(f"Clamped to [{lower_bound:.4f}, {upper_bound:.4f}]")
df_filtered[['log_return']].describe()

Clamped to [-0.0453, 0.0460]


,log_return
count,845248.000000
mean,0.000504
std,0.016835
min,-0.045307
25%,-0.008092
50%,0.000809
75%,0.009499
max,0.046041


In [99]:
from pandas_datareader import data as pdr

ff_raw = pdr.DataReader('F-F_Research_Data_5_Factors_2x3_daily', 'famafrench', start='2015-01-01')[0]

if isinstance(ff_raw.index, pd.PeriodIndex):
    ff_raw.index = ff_raw.index.to_timestamp()

ff = ff_raw.div(100)
ff.columns = ['MKT_RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']

ff

/var/folders/kp/mvqzzmf14fj_byq4km78jych0000gn/T/ipykernel_25527/3983692894.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_raw = pdr.DataReader('F-F_Research_Data_5_Factors_2x3_daily', 'famafrench', start='2015-01-01')[0]


,MKT_RF,SMB,HML,RMW,CMA,RF
Date,,,,,,
2015-01-02,-0.0012,-0.0057,0.0008,-0.0025,0.0012,0.00000
2015-01-05,-0.0184,0.0025,-0.0068,0.0016,-0.0008,0.00000
2015-01-06,-0.0104,-0.0078,-0.0031,0.0053,0.0002,0.00000
2015-01-07,0.0119,0.0017,-0.0066,0.0025,-0.0014,0.00000
2015-01-08,0.0181,-0.0010,-0.0028,0.0011,-0.0016,0.00000
...,...,...,...,...,...,...
2024-12-24,0.0111,-0.0012,-0.0005,-0.0013,-0.0037,0.00017
2024-12-26,0.0002,0.0109,-0.0019,-0.0044,0.0035,0.00017
2024-12-27,-0.0117,-0.0044,0.0056,0.0041,0.0003,0.00017


In [100]:
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
df = df.merge(ff, left_on='date', right_index=True, how='left')

df

,date,TICKER,PERMNO,PRC,log_return,MKT_RF_x,SMB_x,HML_x,RMW_x,CMA_x,...,RMW_y,CMA_y,RF_y,MOM_y,MKT_RF,SMB,HML,RMW,CMA,RF
13962,2015-01-02,ORCL,10104,44.33000,NaN,-0.0012,-0.0057,0.0008,-0.0025,0.0012,...,-0.0025,0.0012,0.0,NaN,-0.0012,-0.0057,0.0008,-0.0025,0.0012,0.0
13963,2015-01-05,ORCL,10104,43.59000,-0.016834,-0.0184,0.0025,-0.0068,0.0016,-0.0008,...,0.0016,-0.0008,0.0,NaN,-0.0184,0.0025,-0.0068,0.0016,-0.0008,0.0
13964,2015-01-06,ORCL,10104,43.14000,-0.010377,-0.0104,-0.0078,-0.0031,0.0053,0.0002,...,0.0053,0.0002,0.0,NaN,-0.0104,-0.0078,-0.0031,0.0053,0.0002,0.0
13965,2015-01-07,ORCL,10104,43.15000,0.000232,0.0119,0.0017,-0.0066,0.0025,-0.0014,...,0.0025,-0.0014,0.0,NaN,0.0119,0.0017,-0.0066,0.0025,-0.0014,0.0
13966,2015-01-08,ORCL,10104,43.41000,0.006007,0.0181,-0.0010,-0.0028,0.0011,-0.0016,...,0.0011,-0.0016,0.0,NaN,0.0181,-0.0010,-0.0028,0.0011,-0.0016,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13967877,2022-03-25,TSLA,93436,1010.64001,-0.003240,0.0027,-0.0031,0.0219,0.0020,0.0133,...,0.0020,0.0133,0.0,NaN,0.0027,-0.0031,0.0219,0.0020,0.0133,0.0
13967878,2022-03-28,TSLA,93436,1091.83997,0.046041,0.0073,-0.0087,-0.0171,0.0050,-0.0119,...,0.0050,-0.0119,0.0,NaN,0.0073,-0.0087,-0.0171,0.0050,-0.0119,0.0
13967879,2022-03-29,TSLA,93436,1099.56995,0.007055,0.0145,0.0120,-0.0194,-0.0034,-0.0098,...,-0.0034,-0.0098,0.0,NaN,0.0145,0.0120,-0.0194,-0.0034,-0.0098,0.0
13967880,2022-03-30,TSLA,93436,1093.98999,-0.005088,-0.0083,-0.0113,0.0042,0.0040,0.0092,...,0.0040,0.0092,0.0,NaN,-0.0083,-0.0113,0.0042,0.0040,0.0092,0.0


In [112]:
import statsmodels.api as sm

df = df.sort_values(['PERMNO','date']).copy()
df['excess_return'] = df['log_return'] - df['RF']



In [113]:
def strip_factors(g):
    y = g['excess_return'].to_numpy()
    X = sm.add_constant(g[['MKT_RF','SMB','HML','RMW','CMA']]).to_numpy()
    resids = np.full(len(g), np.nan)
    for i in range(252, len(g)+1):
        Xi = X[i-252:i]
        yi = y[i-252:i]
        beta, *_ = np.linalg.lstsq(Xi, yi, rcond=None)
        fitted_i = X[i-1] @ beta
        resids[i-1] = y[i-1] - fitted_i
    return pd.Series(resids, index=g.index)

In [114]:
df['idiosyncratic_ret'] = (
    df
      .groupby('PERMNO', group_keys=False)
      .apply(strip_factors)
)

df = df.dropna(subset=['idiosyncratic_ret'])

/var/folders/kp/mvqzzmf14fj_byq4km78jych0000gn/T/ipykernel_25527/2063431334.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(strip_factors)


In [115]:
df.head()

,date,TICKER,PERMNO,PRC,log_return,MKT_RF_x,SMB_x,HML_x,RMW_x,CMA_x,...,RF_y,MOM_y,MKT_RF,SMB,HML,RMW,CMA,RF,excess_return,idiosyncratic_ret
14214,2016-01-04,ORCL,10104,35.75,-0.021584,-0.0159,-0.0076,0.0052,0.0035,0.0042,...,0.0,NaN,-0.0159,-0.0076,0.0052,0.0035,0.0042,0.0,-0.021584,-0.005661
14215,2016-01-05,ORCL,10104,35.64,-0.003082,0.0012,-0.0024,0.0001,0.0005,0.0031,...,0.0,NaN,0.0012,-0.0024,0.0001,0.0005,0.0031,0.0,-0.003082,-0.002864
14216,2016-01-06,ORCL,10104,35.82,0.005038,-0.0135,-0.0023,0.0000,0.0015,0.0003,...,0.0,NaN,-0.0135,-0.0023,0.0000,0.0015,0.0003,0.0,0.005038,0.019179
14217,2016-01-07,ORCL,10104,35.04,-0.022016,-0.0244,-0.0029,0.0008,0.0049,0.0036,...,0.0,NaN,-0.0244,-0.0029,0.0008,0.0049,0.0036,0.0,-0.022016,0.003001
14218,2016-01-08,ORCL,10104,34.65,-0.011193,-0.0111,-0.0052,-0.0003,0.0025,0.0005,...,0.0,NaN,-0.0111,-0.0052,-0.0003,0.0025,0.0005,0.0,-0.011193,0.000047


In [121]:
from scipy.cluster.hierarchy import linkage, fcluster

df = df.sort_values('date')

month_ends = (
    df
      .groupby(df['date'].dt.to_period('M'))['date']
      .max()
      .sort_values()
)

clusters = []
all_dates = df['date'].drop_duplicates().sort_values()

for t in month_ends:
    window_dates = all_dates[all_dates <= t].iloc[-252:]
    window_df = df[df['date'].isin(window_dates)]
    
    # use pivot_table to aggregate duplicates (here we take the mean)
    mat = (
        window_df
          .pivot_table(
             index='date',
             columns='PERMNO',
             values='idiosyncratic_ret',
             aggfunc='mean'      # or 'last', or np.nanmean
          )
          .dropna(axis=1, thresh=200)
          .fillna(method='ffill')
          .fillna(method='bfill')
    )
    
    if mat.shape[1] < 2:
        continue
    
    corr = mat.corr()
    dist = 1 - np.abs(corr)
    
    Z = linkage(dist, method='ward')
    labels = fcluster(Z, t=25, criterion='maxclust')
    
    clusters.append(pd.DataFrame({
        'date': t,
        'PERMNO': corr.index,
        'cluster': labels
    }))

cluster_assignments = pd.concat(clusters, ignore_index=True)

/var/folders/kp/mvqzzmf14fj_byq4km78jych0000gn/T/ipykernel_25527/1341645405.py:21: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  window_df
/var/folders/kp/mvqzzmf14fj_byq4km78jych0000gn/T/ipykernel_25527/1341645405.py:21: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  window_df
/var/folders/kp/mvqzzmf14fj_byq4km78jych0000gn/T/ipykernel_25527/1341645405.py:21: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  window_df
/var/folders/kp/mvqzzmf14fj_byq4km78jych0000gn/T/ipykernel_25527/1341645405.py:21: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  window_df
/var/folders/kp/mvqzzmf14fj_byq4km78jych0000gn/T/ipykernel_25527/1341645405.

In [122]:
cluster_assignments.head()

,date,PERMNO,cluster
0,2016-10-31,10104,10
1,2016-10-31,10107,8
2,2016-10-31,10138,10
3,2016-10-31,10145,10
4,2016-10-31,10516,10


In [125]:
cluster_assignments = cluster_assignments.merge(
    df[['PERMNO','TICKER']].drop_duplicates(),
    on='PERMNO',
    how='left'
)

date = cluster_assignments['date'].max()
members = (
    cluster_assignments[cluster_assignments['date'] == date]
      .groupby('cluster')['TICKER']
      .apply(list)
)
print(members)

clusters_by_date = {
    d: grp.groupby('cluster')['TICKER'].apply(list).to_dict()
    for d, grp in cluster_assignments.groupby('date')
}

import pprint

pprint.pprint(clusters_by_date[date])

cluster
1                  [VICI, FRT, VTR, KIM, REG, SPG, BXP]
2     [DOC, INVH, UDR, PSA, CPT, EQR, MAA, AVB, ESS,...
3     [KO, PEP, KHC, HSY, GIS, KMB, PG, CL, CHD, CPB...
4     [ED, DTE, EIX, SO, EXC, CNP, PPL, FE, CMS, WEC...
5     [XOM, TRGP, KMI, MPC, PSX, FANG, COP, SLB, CVX...
6     [CFG, NEM, FITB, RF, MTB, WFC, HBAN, PNC, KEY,...
7     [SYF, JPM, BK, AFL, NTRS, AXP, BAC, GL, AIG, M...
8                             [LEN, PHM, DHI, NVR, LEN]
9     [MAS, SHW, WY, JBHT, SWK, GPC, TGT, HAS, SNA, ...
10    [GE, BA, HWM, DIS, SYY, DRI, GPN, FIS, TDG, MA...
11    [GM, IBM, APTV, NWSA, NWS, MTCH, MRNA, FOXA, F...
12    [TROW, HON, ADM, LYB, VMC, DD, LIN, DOW, CAT, ...
13             [CDNS, KEYS, TRMB, ZBRA, SNPS, APH, TEL]
14    [ORCL, FICO, KKR, APO, EPAM, ENPH, NOW, PANW, ...
15    [NXPI, AMAT, TXN, SWKS, KLAC, LRCX, TER, MU, A...
16                  [GD, HII, RTX, LMT, NOC, LHX, LDOS]
17    [HCA, MDT, COO, SYK, BSX, STE, UHS, RMD, EW, I...
18    [PCG, VST, WRB, CINF, L, UNP, TRV,

In [126]:
date_of_interest = cluster_assignments['date'].max()  # or set to any date you want
counts_for_date = (
    cluster_assignments[cluster_assignments['date'] == date_of_interest]
      .groupby('cluster')
      .size()
      .sort_index()
)
print(counts_for_date)

# If you want counts for every date and cluster
counts_all = (
    cluster_assignments
      .groupby(['date','cluster'])
      .size()
      .reset_index(name='n_members')
      .sort_values(['date','cluster'])
)
print(counts_all)

cluster
1      7
2     19
3     17
4     26
5     19
6     11
7     22
8      5
9     31
10    11
11    36
12    32
13     7
14    54
15    16
16     7
17    12
18    14
19    25
20    18
21    15
22    26
23    15
24    14
25    18
dtype: int64
           date  cluster  n_members
0    2016-10-31        1         15
1    2016-10-31        2         22
2    2016-10-31        3         25
3    2016-10-31        4         15
4    2016-10-31        5          4
...         ...      ...        ...
1645 2022-03-31       21         15
1646 2022-03-31       22         26
1647 2022-03-31       23         15
1648 2022-03-31       24         14
1649 2022-03-31       25         18

[1650 rows x 3 columns]


In [ ]:
import numpy as np
import pandas as pd
from itertools import combinations

def backtest_pairs(df, pairs, entry_z=1.0, exit_z=0.0):
    """
    df: DataFrame indexed by date, columns = PERMNO, values = idiosyncratic_ret
    pairs: list of (i,j) tuples of PERMNOs
    returns: pnl_series (daily), trades_summary DataFrame
    """
    pnl = pd.Series(0.0, index=df.index)
    trades = []
    
    for i,j in pairs:
        spread = df[i] - df[j]
        z = (spread - spread.rolling(252).mean()) / spread.rolling(252).std()
        
        position = 0
        entry_price = 0
        
        for t in df.index[252:]:
            if position == 0:
                if z.loc[t] > entry_z:
                    position = -1
                    entry_price = spread.loc[t]
                elif z.loc[t] < -entry_z:
                    position = +1
                    entry_price = spread.loc[t]
            else:
                if abs(z.loc[t]) < exit_z:
                    ret = position * (spread.loc[t] - entry_price)
                    trades.append({
                        'pair': (i,j),
                        'entry_date': entry_date,
                        'exit_date': t,
                        'entry_z': z.loc[entry_date],
                        'exit_z': z.loc[t],
                        'pnl': ret
                    })
                    position = 0
            pnl.loc[t] += position * (spread.diff().loc[t])
            if position != 0 and position * spread.loc[t] < position * entry_price:
                entry_date = entry_date if 'entry_date' in locals() else t
    pnl = pnl.fillna(0)
    trades_df = pd.DataFrame(trades)
    return pnl, trades_df


idio = (
    df
      .pivot_table(index='date', columns='PERMNO', values='idiosyncratic_ret')
      .dropna(axis=1, thresh=200)
)

last_date = cluster_assignments['date'].max()
permnos = cluster_assignments.query("date==@last_date").groupby('cluster')['PERMNO'].apply(list)

cluster_pairs = []
for clist in permnos:
    if len(clist) < 2: continue
    cluster_pairs += list(combinations(clist, 2))

pnl_cluster, trades_cluster = backtest_pairs(idio, cluster_pairs)

benchmark_tickers = ['AAPL','MSFT']
bench = (
    df
      .query("TICKER in @benchmark_tickers")
      .pivot(index='date', columns='TICKER', values='log_return')
      .dropna()
)
bench_pairs = [(bench.columns[0], bench.columns[1])]
pnl_bench, trades_bench = backtest_pairs(bench, bench_pairs)

def perf_stats(pnl, freq=252):
    ret = pnl.sum()
    vol = pnl.std() * np.sqrt(freq)
    sr = pnl.mean() / pnl.std() * np.sqrt(freq)
    return {'annual_return': ret, 'annual_vol': vol, 'sharpe': sr}

cluster_stats = perf_stats(pnl_cluster)
bench_stats   = perf_stats(pnl_bench)

print("Cluster Pairs Strategy:", cluster_stats)
print("Benchmark Pair:", bench_stats)
